# Wstęp
Zadanie 9 stanowi pierwszy z trzech etapów zajęć poświęconych sieciom rekurencyjnym i predykcji z wykorzystaniem danych multimodalnych. Efektem wszystkich trzech etapów będzie sieć rekurencyjna z warstwą atencji do predykcji kursu kryptowaluty Bitcoin (BTC) w oparciu o dane z giełdy oraz o wyniki analizy emocji komunikatów z mediów społecznościowych, do których również należy utworzyć dedykowany model sieci rekurencyjnej. Plan realizacji etapów wygląda następująco:

1.   EmoTweet - model sieci rekurencyjnej do analizy emocji 
2.   MultiBTC - multimodalny model sieci rekurencyjnej do predykcji kursu BTC
3.   AttEmoTweet & AttMultiBTC - rozszerzenie modeli EmoTweet i MultiBTC o warstwę atencji 

Każdy etap jest traktowany jako oddzielna lista na laboratorium, za którą można otrzymać 10 punktów. 

# Cel ćwiczenia

Celem pierwszego etapu prac jest zapoznanie się z podstawową siecią rekurencyjną LSTM. Ze względu na fakt, że model ten będzie wykorzystany do analizy emocji tekstu, w ramach teorii do zadania zostanie omówiony podstawowy mechanizm konwersji słów w tekście do postaci wektorów dystrybucyjnych (tzw. word embeddings) na podstawie rozwiązania o nazwie `fastText`. Modele będą budowane na ogólnodostępnym zbiorze `TweetEval`, zawierającym podzbiory ręcznie anotowanych tweetów przy pomocy etykiet odnoszących się do następujących zjawisk: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

# Warunki zaliczenia

Do zaliczenia pierwszego etapu należy utworzyć następujące modele dla min. 2 wybranych zjawisk:

1.   Model bazowy (regresja logistyczna).
2.   Model rekurencyjny oparty o sieć LSTM.

Wytrenowane modele będą wykorzystane w 2 etapie, dlatego proszę je zachować.

# Wektory dystrybucyjne

W przetwarzaniu języka naturalnego, o wektorach dystrybucyjnych (inaczej osadzeniach lub zanurzeniach, ang. word embeddings) mówi się w kontekście reprezentacji słów w tekście, zazwyczaj w postaci wektora liczb rzeczywistych, który koduje znaczenie słowa. Hipoteza dystrybucyjna, u podstawy której leży większość metod reprezentacji, mówi o tym, że słowa, które często współwystępują, mają podobne znaczenie. Wektory dystrybucyjne można uzyskać za pomocą zestawu technik modelowania języka, w których słowa lub frazy są mapowane do wektorów liczb rzeczywistych. Z reguły polega to na matematycznym zanurzeniu z przestrzeni o wielu wymiarach opisujących słowo (konteksty) do ciągłej przestrzeni wektorowej o znacznie mniejszym wymiarze.

Metody generowania tego odwzorowania obejmują sieci neuronowe, redukcję wymiarowości na macierzy współwystępowania słów, modele probabilistyczne lub jawną reprezentację w kontekście, w którym pojawiają się słowa. Wektory dystrybucyjne, używane jako podstawowa reprezentacja wejściowa tekstu, okazały się istotnie poprawiać jakość w wielu zadaniach NLP, takich jak np. rozpoznawanie nazw własnych, określanie części mowy, rozpoznawanie dziedziny tekstu, czy też rozpoznawanie wydźwięku i emocji w tekście. 

# fastText

[fastText](https://fasttext.cc/) jest biblioteką do efektywnego uczenia modeli reprezentacji wektorowych słów oraz do budowania klasyfikatorów tekstu. Modele językowe można budować z wykorzystaniem dwóch popularnych technik: [Continuous Bag of Words](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-cbow.html) oraz [Skip-Gram](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c). 

## Instalacja

Pobranie repozytorium projektu:


In [1]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 25.52 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.


Instalacja biblioteki:

In [2]:
!cd fastText && mkdir build && cd build && cmake ..  && make && make install

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /content/fastText/build
Scanning dependencies of target fasttext-static_pic
[  2%] Building CXX object CMakeFiles/fasttext-static_pic.dir/src/args.cc.o
[  4%] Building CXX object CMakeFiles/fasttext-static_pic.dir/src/autotune.cc.o
[  6%] Building CXX object CMakeFiles/fasttext-static_pic.dir/src/densematrix.cc.o
[  8%] Building

Instalacja API do Pythona:

In [3]:
!cd fastText && pip install .

Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3092562 sha256=8ce36611a7846e060b8fe9be75cc2dcb66d30ae3410073e280b481a38aadab55
  Stored in directory: /tmp/pip-ephem-wheel-cache-_x9uxvkz/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


# Dane do etapu nr 1

## Korpus 
Korpus (zbiór dokumentów) do realizacji etapu nr 1 pochodzą z repozytorium [TweetEval](https://github.com/cardiffnlp/tweeteval). Repozytorium zawiera 7 różnorodnych zbiorów danych, zawierających zanonimizowane wpisy z [Twittera](https://twitter.com), anotowane następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

In [4]:
!wget http://jankocon.clarin-pl.eu/share/tweeteval.7z

--2021-05-05 07:20:21--  http://jankocon.clarin-pl.eu/share/tweeteval.7z
Resolving jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)... 156.17.135.34
Connecting to jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)|156.17.135.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17390348 (17M) [application/x-7z-compressed]
Saving to: ‘tweeteval.7z’

tweeteval.7z        100%[===================>]  16.58M   604 B/s    in 28s     

2021-05-05 07:20:49 (615 KB/s) - ‘tweeteval.7z’ saved [17390348/17390348]



In [5]:
!7za x tweeteval.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 17390348 bytes (17 MiB)

Extracting archive: tweeteval.7z
--
Path = tweeteval.7z
Type = 7z
Physical Size = 17390348
Headers Size = 1810
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     59% 66 - tweeteval/datasets/emoji/val_labels.txt                                                 Everything is Ok

Folders: 33
Files: 115
Size:       30563155
Compressed: 17390348


## Zawartość korpusu

W katalogu głównym (tweeteval) znajdują się następujące elementy:
*   `datasets` - katalog ze zbiorami danych
   * `emotion` - tweety anotowane emocjami 
     * `mapping.txt` - identyfikatory etykiet oraz ich opis
     * `train_text.txt` - wpisy z Twittera (część ucząca)
     * `train_labels.txt` - etykiety wpisów z Twittera (część ucząca)
     * `test_*.txt, valid_*.txt` - j.w. (część testowa i walidacyjna)
   * `emoji` - tweety anotowane emotikonami
   * `...` - katalogi zawierające tweety anotowane pozostałymi zjawiskami
*   `predictions` - katalog z przykładowymi predykcjami
   * `emotion.txt` - etykiety modelu predykcyjnego dla części testowej danych `emotion`
   * `emoji.txt` - j.w. dla cz. testowej danych `emoji`
   * `...` - j.w. dla pozostałych danych
*   `evaluation_script.py` - skrypt do ewaluacji 

## Model języka

Na potrzeby zadania został przygotowany model Skip-Gram reprezentacji wektorowej słów, zbudowany na wielkim korpusie tweetów dotyczących kursu BTC. Wersja binarna tego modelu dostępna jest w 2 wariantach:
* [wektory 100-elementowe (1.7GB)](http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_100_en.bin)
* [wektory 20-elementowe (350MB)](http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin)

Na potrzeby prezentacji przykładowego rozwiązania zostanie wykorzystany mniejszy model. Do realizacji ostatecznego rozwiązania należy wykorzystać większy model. 





# Model bazowy rozpoznawania emocji

Model bazowy, zbudowany z wykorzystaniem narzędzia fastText (oparty o regresję logistyczną), będzie punktem wyjścia do porównania się z modelami opartymi o sieci LSTM, których skonstruowanie i ewaluacja na wybranych zadaniach będzie celem etapu nr 1. 

Pobranie mniejszego modelu reprezentacji języka tweetów:


In [6]:
!wget http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin

--2021-05-05 07:20:50--  http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin
Resolving jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)... 156.17.135.34
Connecting to jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)|156.17.135.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365240858 (348M) [application/octet-stream]
Saving to: ‘fasttext_tweetmodel_btc_sg_20_en.bin’

fasttext_tweetmodel 100%[===================>] 348.32M  11.2MB/s    in 41s     

2021-05-05 07:21:31 (8.49 MB/s) - ‘fasttext_tweetmodel_btc_sg_20_en.bin’ saved [365240858/365240858]



In [85]:
!wget http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_100_en.bin

--2021-05-05 08:14:54--  http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_100_en.bin
Resolving jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)... 156.17.135.34
Connecting to jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)|156.17.135.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1740570138 (1.6G) [application/octet-stream]
Saving to: ‘fasttext_tweetmodel_btc_sg_100_en.bin’

fasttext_tweetmodel 100%[===================>]   1.62G  11.2MB/s    in 2m 30s  

2021-05-05 08:17:24 (11.1 MB/s) - ‘fasttext_tweetmodel_btc_sg_100_en.bin’ saved [1740570138/1740570138]



Wydobycie słownika wektorów z binarnego modelu języka:

In [7]:
!python fastText/python/doc/examples/bin_to_vec.py fasttext_tweetmodel_btc_sg_20_en.bin > fasttext_tweetmodel_btc_sg_20_en.vec

In [96]:
!python fastText/python/doc/examples/bin_to_vec.py fasttext_tweetmodel_btc_sg_100_en.bin > fasttext_tweetmodel_btc_sg_100_en.vec

tcmalloc: large alloc 1259585536 bytes == 0x561c70112000 @  0x7f0e4f398887 0x7f0e4d2cf733 0x7f0e4d2b83cf 0x7f0e4d2b89e1 0x7f0e4d280273 0x7f0e4d2a4e0c 0x561c58171c38 0x561c581e563d 0x561c581dfb0e 0x561c5817302c 0x561c581b3d39 0x561c581b0c84 0x561c581718e9 0x561c581e5ade 0x561c5817269a 0x561c581e0a45 0x561c581dfb0e 0x561c581df813 0x561c582a9592 0x561c582a990d 0x561c582a97b6 0x561c58281103 0x561c58280dac 0x7f0e4e180bf7 0x561c58280c8a


Dodanie prefiksu `__label__` do etykiet zbioru `emotion`:

In [8]:
!sed 's/^/__label__/g' tweeteval/datasets/emotion/train_labels.txt > train_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/test_labels.txt > test_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/val_labels.txt > val_labels_emo.txt

In [93]:
!sed 's/^/__label__/g' tweeteval/datasets/offensive/train_labels.txt > train_labels_off.txt
!sed 's/^/__label__/g' tweeteval/datasets/offensive/test_labels.txt > test_labels_off.txt
!sed 's/^/__label__/g' tweeteval/datasets/offensive/val_labels.txt > val_labels_off.txt

Przygotowanie zbioru uczącego, testowego i walidacyjnego w formacie `fastText`:

In [9]:
!paste -d " " tweeteval/datasets/emotion/train_text.txt train_labels_emo.txt > train_emo.txt
!paste -d " " tweeteval/datasets/emotion/test_text.txt test_labels_emo.txt > test_emo.txt
!paste -d " " tweeteval/datasets/emotion/val_text.txt val_labels_emo.txt > val_emo.txt

In [94]:
!paste -d " " tweeteval/datasets/offensive/train_text.txt train_labels_off.txt > train_off.txt
!paste -d " " tweeteval/datasets/offensive/test_text.txt test_labels_off.txt > test_off.txt
!paste -d " " tweeteval/datasets/offensive/val_text.txt val_labels_off.txt > val_off.txt

Trenowanie modelu z wykorzystaniem wejścia `train_emo.txt`, z określeniem wyjściowej nazwy modelu `emo_model`, dla wektorów słów o wymiarze `20`, z wykorzystaniem pretrenowanych wektorów z pliku `fasttext_tweetmodel_btc_sg_20_en.vec` i z uruchomieniem dostrajania hiperparametrów na zbiorze walidacyjnym `val_emo.txt`:

In [10]:
!fasttext supervised -input train_emo.txt -output emo_model -dim 20 -pretrainedVectors fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_emo.txt 

Progress: 100.0% Trials:   21 Best score:  0.689840 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  12887
Number of labels: 4
Progress: 100.0% words/sec/thread:   77263 lr:  0.000000 avg.loss:  0.538675 ETA:   0h 0m 0s


Podstawowa ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision) i kompletność (R - recall) w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [11]:
!fasttext test emo_model.bin test_emo.txt

N	1421
P@1	0.697
R@1	0.697


Rozszerzona ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision), kompletność (R - recall) oraz F1-score dla każdej etykiety w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [12]:
!fasttext test-label emo_model.bin test_emo.txt

F1-Score : 0.765612  Precision : 0.699259  Recall : 0.845878   __label__0
F1-Score : 0.692308  Precision : 0.701613  Recall : 0.683246   __label__3
F1-Score : 0.662595  Precision : 0.730640  Recall : 0.606145   __label__1
F1-Score : 0.400000  Precision : 0.519481  Recall : 0.325203   __label__2
N	1421
P@1	0.697
R@1	0.697


Przygotowanie danych do ewaluacji z wykorzystaniem skryptu dołączonego do zbioru TweetEval:

In [13]:
!mkdir predictions2

In [14]:
!fasttext predict emo_model.bin tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions2/emotion.txt

Uruchomienie ewaluacji. Oprócz wyników P, R, F1 [weighted]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)) dla każdej etykiety, otrzymujemy również wyniki w wariancie [macro]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)). **Ostateczną miarą (TweetEval Score) jest miara F1-score w wariancie macro i tę miarę proszę traktować jako kluczową przy porównywaniu rozwiązań.**

In [15]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task emotion

0 {'precision': 0.6992592592592592, 'recall': 0.8458781362007168, 'f1-score': 0.7656123276561233, 'support': 558}
1 {'precision': 0.7306397306397306, 'recall': 0.6061452513966481, 'f1-score': 0.6625954198473283, 'support': 358}
2 {'precision': 0.5194805194805194, 'recall': 0.3252032520325203, 'f1-score': 0.39999999999999997, 'support': 123}
3 {'precision': 0.7016129032258065, 'recall': 0.6832460732984293, 'f1-score': 0.6923076923076923, 'support': 382}
accuracy 0.6966924700914848
macro avg {'precision': 0.6627481031513289, 'recall': 0.6151181782320787, 'f1-score': 0.6301288599527859, 'support': 1421}
weighted avg {'precision': 0.6922363991302268, 'recall': 0.6966924700914848, 'f1-score': 0.6883056844468676, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6301288599527859


# Budowa modeli EmoTweet

W tej sekcji Państwa zadaniem będzie przygotowanie modeli sieci LSTM oraz modeli bazowych opartych o regresję logistyczną (fastText) dla wybranych 2 zjawisk ze zbioru TweetEval. Dla sieci LSTM kolejne jednostki sieci rekurencyjnej na wejściu dostają reprezentację wektorową kolejnych wyrazów w tekście. Wyjście z ostatniej jednostki podlega klasyfikacji. W celu usprawnienia zadania, przedstawiona zostanie metoda reprezentacji wektorowej tekstu z wykorzystaniem Pythonowego API do narzędzia fastText. Do ewaluacji modeli należy wykorzystać uprzednio zaprezentowany skrypt `tweeteval/evaluation_script.py`.

## Wektoryzacja tekstu


In [16]:
# inicjalizacja biblioteki
import fasttext

In [87]:
# ładowanie modelu
MODEL_PATH = 'fasttext_tweetmodel_btc_sg_100_en.bin'
model = fasttext.load_model(MODEL_PATH)

In [100]:
# wczytanie danych treningowych
import pandas as pd
TRAIN_PATH_EMO = 'tweeteval/datasets/emotion/train_text.txt'
VAL_PATH_EMO = 'tweeteval/datasets/emotion/val_text.txt'
TEST_PATH_EMO = 'tweeteval/datasets/emotion/test_text.txt'

TRAIN_PATH_OFF = 'tweeteval/datasets/offensive/train_text.txt'
VAL_PATH_OFF = 'tweeteval/datasets/offensive/val_text.txt'
TEST_PATH_OFF = 'tweeteval/datasets/offensive/test_text.txt'

In [19]:
import torch
import numpy as np

In [88]:
def prepare_lstm_data(tweets_path, label_path):
  texts = pd.read_csv(tweets_path, sep='\t', header=None)
  labels = pd.read_csv(label_path, sep='\t', header=None)

  lengths = torch.tensor([len(fasttext.tokenize(x)) for x in texts[0]])

  assert len(texts) == len(labels)

  training_data = []

  for tweet in texts[0].tolist():
    tweet_emb = []
    for word in fasttext.tokenize(tweet):
      tweet_emb.append(torch.tensor(model.get_word_vector(word).reshape(1, -1)))
  
    training_data.append(torch.cat(tweet_emb))
  training_data_padded = torch.nn.utils.rnn.pad_sequence(training_data, batch_first = True)
  print(training_data_padded.shape)

  labels_ints = torch.tensor([int(str_label[-1]) for  str_label in labels[0]])

  return training_data_padded, lengths, labels_ints




In [103]:
X_train_emo, train_lengths_emo, y_train_emo = prepare_lstm_data(TRAIN_PATH_EMO, 'train_labels_emo.txt')
X_val_emo, val_lengths_emo, y_val_emo = prepare_lstm_data(VAL_PATH_EMO, 'val_labels_emo.txt')
X_test_emo, test_lengths_emo, y_test_emo = prepare_lstm_data(TEST_PATH_EMO, 'test_labels_emo.txt')

X_train_off, train_lengths_off, y_train_off = prepare_lstm_data(TRAIN_PATH_OFF, 'train_labels_off.txt')
X_val_off, val_lengths_off, y_val_off = prepare_lstm_data(VAL_PATH_OFF, 'val_labels_off.txt')
X_test_off, test_lengths_off, y_test_off = prepare_lstm_data(TEST_PATH_OFF, 'test_labels_off.txt')


torch.Size([3257, 33, 100])
torch.Size([374, 32, 100])
torch.Size([1421, 36, 100])
torch.Size([11916, 103, 100])
torch.Size([1324, 82, 100])
torch.Size([860, 63, 100])


In [104]:
from torch.utils.data import DataLoader, TensorDataset
train_dataloader_emo = DataLoader(TensorDataset(X_train_emo, train_lengths_emo, y_train_emo), batch_size=100, shuffle=True)
val_dataloader_emo = DataLoader(TensorDataset(X_val_emo, val_lengths_emo,  y_val_emo), batch_size = 100)

train_dataloader_off = DataLoader(TensorDataset(X_train_off, train_lengths_off, y_train_off), batch_size=100, shuffle=True)
val_dataloader_off = DataLoader(TensorDataset(X_val_off, val_lengths_off,  y_val_off), batch_size = 100)


# New Section

Proszę zwrócić uwagę, że fastText jest w stanie przyporządkować reprezentację wektorową nawet dla takich słów, których model języka nie widział w trakcie uczenia (pierwszy token wejściowego tekstu). 

## Model klasyfikacji tekstu LSTM

In [25]:
from torch import nn

In [79]:
class LSTMNet(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, classes):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden = nn.Linear(hidden_dim, 50)
        self.hidden2out = nn.Linear(50, classes)


    def forward(self, sentence, lengths):
        sentence = torch.nn.utils.rnn.pack_padded_sequence(sentence, lengths, enforce_sorted=False, batch_first=True)
        _, (hn, cn) = self.lstm(sentence)
        hidden = torch.nn.functional.sigmoid(self.hidden(hn.squeeze(dim=0)))
        return self.hidden2out(hidden)


## Trenowanie modeli LSTM dla ZJAWISKO_1 i ZJAWISKO_2

In [ ]:
from tqdm.auto import tqdm
def validate(
    model: nn.Module, 
    loss_fn: torch.nn.CrossEntropyLoss, 
    dataloader: DataLoader):
    loss = 0
    correct = 0
    all = 0
    for X_batch, lengths, y_batch in tqdm(dataloader):
        y_pred = model(X_batch, lengths)
        all += len(y_pred)
        loss += loss_fn(y_pred, y_batch).sum()
        correct += count_correct(y_pred, y_batch)
    return loss / len(dataloader), correct / all

def count_correct(
    y_pred: torch.Tensor, y_true: torch.Tensor
) -> torch.Tensor:
    preds = torch.argmax(y_pred, dim=1)
   
    return (preds == y_true).float().sum()

def fit(
    model: nn.Module, optimiser,
    loss_fn: torch.nn.CrossEntropyLoss, train_dl: DataLoader, 
    val_dl: DataLoader, epochs: int, 
    print_metrics: str = True
):

    best_val_loss = float('inf') 
    for epoch in range(epochs):
        print(epoch)
        model.train()

        for X_batch, lengths,  y_batch in tqdm(train_dl):
            y_pred = model(X_batch, lengths) # Uzyskanie pseudoprawdopodobieństw dla próbek z minibatcha
            loss = loss_fn(y_pred, y_batch) # Policzenie funkcji straty
            loss.backward() # Wsteczna propagacja z wyniku funkcji straty - policzenie gradientów i zapisanie ich w tensorach (parametrach)
            optimiser.step() # Aktualizacja parametrów modelu przez optymalizator na podstawie gradientów zapisanych w tensorach (parametrach) oraz lr
            optimiser.zero_grad() # Wyzerowanie gradientów w modelu, alternatywnie można wywołać percepron.zero_grad()
        
        if print_metrics: 
            model.eval() # Przełączenie na tryb ewaluacji modelu - istotne dla takich warstw jak Dropuot czy BatchNorm
            with torch.no_grad():  # Wstrzymujemy przeliczanie i śledzenie gradientów dla tensorów - w procesie ewaluacji modelu nie chcemy zmian w gradientach
                train_loss, train_acc = validate(model, loss_fn, train_dl)
                val_loss, val_acc = validate(model, loss_fn, val_dl)
                print(train_acc, val_acc)


      
    model.eval() # Przełączenie na tryb ewaluacji modelu - istotne dla takich warstw jak Dropuot czy BatchNorm


In [ ]:
net_emo = LSTMNet(100, 200, 4)
optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
fit(
    model=net, optimiser=optimiser, loss_fn=loss_fn, 
    train_dl=train_dataloader_emo, val_dl=val_dataloader_emo, epochs=30
)


In [ ]:
net_off = LSTMNet(100, 200, 2)
optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
fit(
    model=net, optimiser=optimiser, loss_fn=loss_fn, 
    train_dl=train_dataloader_off, val_dl=val_dataloader_off, epochs=100
)

0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



tensor(0.6693) tensor(0.6533)
1


## Trenowanie modeli LR (fastText) dla ZJAWISKO_1 i ZJAWISKO_2

In [ ]:
!fasttext supervised -input train_emo.txt -output emo_model -dim 100 -pretrainedVectors fasttext_tweetmodel_btc_sg_100_en.vec -autotune-validation val_emo.txt 

In [99]:
!fasttext supervised -input train_off.txt -output off_model -dim 100 -pretrainedVectors fasttext_tweetmodel_btc_sg_100_en.vec -autotune-validation val_off.txt 

Progress:  16.0% Trials:    1 Best score:   unknown ETA:   0h 4m11stcmalloc: large alloc 1263263744 bytes == 0x561e9600a000 @  0x7f127a47f887 0x561e5b677b9d 0x561e5b68fea1 0x561e5b6938be 0x561e5b67309f 0x561e5b662794 0x561e5b65e127 0x7f127951cbf7 0x561e5b65e66a
Progress:  84.7% Trials:    5 Best score:  0.741692 ETA:   0h 0m45stcmalloc: large alloc 1901199360 bytes == 0x561ee804a000 @  0x7f127a47f887 0x561e5b677b9d 0x561e5b68fea1 0x561e5b6938be 0x561e5b67309f 0x561e5b662794 0x561e5b65e127 0x7f127951cbf7 0x561e5b65e66a
Progress: 100.0% Trials:    6 Best score:  0.756798 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  36464
Number of labels: 2
Progress: 100.0% words/sec/thread:   73660 lr:  0.000000 avg.loss:  0.176918 ETA:   0h 0m 0s


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_1

In [108]:
!fasttext predict emo_model.bin tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions2/emotion.txt
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task emotion

0 {'precision': 0.6992592592592592, 'recall': 0.8458781362007168, 'f1-score': 0.7656123276561233, 'support': 558}
1 {'precision': 0.7306397306397306, 'recall': 0.6061452513966481, 'f1-score': 0.6625954198473283, 'support': 358}
2 {'precision': 0.5194805194805194, 'recall': 0.3252032520325203, 'f1-score': 0.39999999999999997, 'support': 123}
3 {'precision': 0.7016129032258065, 'recall': 0.6832460732984293, 'f1-score': 0.6923076923076923, 'support': 382}
accuracy 0.6966924700914848
macro avg {'precision': 0.6627481031513289, 'recall': 0.6151181782320787, 'f1-score': 0.6301288599527859, 'support': 1421}
weighted avg {'precision': 0.6922363991302268, 'recall': 0.6966924700914848, 'f1-score': 0.6883056844468676, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6301288599527859


In [ ]:
#TODO

## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_2

In [107]:
!fasttext predict off_model.bin tweeteval/datasets/offensive/test_text.txt | sed 's/__label__//g' > predictions2/offensive.txt
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task offensive

0 {'precision': 0.7970027247956403, 'recall': 0.9435483870967742, 'f1-score': 0.8641063515509603, 'support': 620}
1 {'precision': 0.7222222222222222, 'recall': 0.37916666666666665, 'f1-score': 0.4972677595628416, 'support': 240}
accuracy 0.786046511627907
macro avg {'precision': 0.7596124735089312, 'recall': 0.6613575268817204, 'f1-score': 0.6806870555569009, 'support': 860}
weighted avg {'precision': 0.776133747333291, 'recall': 0.786046511627907, 'f1-score': 0.7617327909961364, 'support': 860}
------------------------------
TweetEval Score (offensive): 0.6806870555569009
